In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
# import dummy classifier
from sklearn.dummy import DummyClassifier


In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
import pandas as pd

# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0]
df_grouped_meal = df_meal.groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'

In [ ]:
# Reset the index
df_play = df_meal.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0]
df_grouped_play = df_play.groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_play.index.name = 's_id'

In [ ]:
# Reset the index of the grouped dataframes
df_grouped_meal = df_grouped_meal.reset_index()
df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'

# Concatenate the DataFrames
final_df = pd.concat([df_grouped_meal, df_grouped_play], axis=0)

final_df

In [ ]:
final_df = final_df.drop(columns=['index'])
final_df.set_index('s_id', inplace=True)
final_df


In [ ]:
#drop last column from final_df
final_df = final_df.drop(columns=['index'])
final_df.set_index('s_id', inplace=True)
final_df


In [ ]:
final_df.set_index('group', append=True, inplace=True)

In [ ]:
final_df

In [ ]:
# Reset the index
final_df.reset_index(inplace=True)

# Convert the first level of the index to integers
final_df.iloc[:, 0] = final_df.iloc[:, 0].astype(int)

# Set the index again
final_df.set_index(final_df.columns[:2].tolist(), inplace=True)
final_df

In [ ]:
speech = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/speech/classification.xlsx', index_col=0)

In [ ]:
# set index name to s_id
speech.index.name = 's_id'
# remove age column from speech
speech = speech.drop(columns=['age'])

In [ ]:
# Create a boolean mask where True indicates a duplicated index
mask = speech.index.duplicated(keep='first')

# Use np.where to assign 'meal' to the first occurrence and 'play' to the second
speech['group'] = np.where(mask, 'play', 'meal')

In [ ]:
speech.set_index('group', append=True, inplace=True)

In [ ]:
## Separate additions to speech features 

to_add = final_df[['child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
speech[to_add.columns] = to_add

In [ ]:
speech['label'] = speech['label'].map({'no_rad': 0, 'rad': 1})


In [ ]:
speech = speech.reset_index()
# set s_id as index for final_df
speech.set_index('s_id', inplace=True)
speech

In [ ]:
# drop nan rows from speech
speech = speech.dropna()

In [ ]:


# Perform a grid search for each classifier
X = speech.drop(['label','group'], axis=1)
y = speech['label']
groups = speech.index

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {'lr__C': [0.01, 0.1, 1, 10, 100], 'lr__penalty': ['l1', 'l2'], 'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {'svc_linear__C': [0.01, 0.1, 1, 10, 100]}),
    ('svc_rbf', SVC(kernel='rbf'), {'svc_rbf__C': [0.01, 0.1, 1, 10, 100], 'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]}),
    ('rf', RandomForestClassifier(), {'rf__n_estimators': [10, 50, 100, 200], 'rf__max_depth': [None, 5, 10, 15], 'rf__min_samples_split': [2, 5, 10]})
]
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled,n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled, n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled, n_jobs=-1)

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
## Try out movement features separately 
X = final_df.reset_index()
# set s_id as index for final_df
X.set_index('s_id', inplace=True)

X = X[['child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y = final_df['label']
groups = X.index

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {'lr__C': [0.01, 0.1, 1, 10, 100], 'lr__penalty': ['l1', 'l2'], 'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {'svc_linear__C': [0.01, 0.1, 1, 10, 100]}),
    ('svc_rbf', SVC(kernel='rbf'), {'svc_rbf__C': [0.01, 0.1, 1, 10, 100], 'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]}),
    ('rf', RandomForestClassifier(), {'rf__n_estimators': [10, 50, 100, 200], 'rf__max_depth': [None, 5, 10, 15], 'rf__min_samples_split': [2, 5, 10]})
]
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled,n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled, n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled, n_jobs=-1)

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()